In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
df=pd.read_csv('week4/data/flight_prices.csv')
df.head()


,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,AirAsia,I5-747,Delhi,Evening,one,Early_Morning,Mumbai,Economy,12.25,1,5949
1,AirAsia,I5-747,Delhi,Evening,one,Morning,Mumbai,Economy,16.33,1,5949
2,GO_FIRST,G8-266,Delhi,Early_Morning,one,Evening,Mumbai,Economy,11.75,1,5954
3,GO_FIRST,G8-101,Delhi,Early_Morning,one,Night,Mumbai,Economy,14.50,1,5954
4,GO_FIRST,G8-103,Delhi,Evening,one,Morning,Mumbai,Economy,15.67,1,5954


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300153 entries, 0 to 300152
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   airline           300153 non-null  object 
 1   flight            300153 non-null  object 
 2   source_city       300153 non-null  object 
 3   departure_time    300153 non-null  object 
 4   stops             300153 non-null  object 
 5   arrival_time      300153 non-null  object 
 6   destination_city  300153 non-null  object 
 7   class             300153 non-null  object 
 8   duration          300153 non-null  float64
 9   days_left         300153 non-null  int64  
 10  price             300153 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 25.2+ MB


In [8]:
df1=df.groupby(['flight','airline'],as_index=False).count()
df1.airline.value_counts()

airline
Indigo       704
Air_India    218
GO_FIRST     205
SpiceJet     186
Vistara      133
AirAsia      115
Name: count, dtype: int64

In [9]:
df.groupby(['flight','source_city','destination_city','airline','class'],as_index=False).count().groupby(['source_city','destination_city'],as_index=False)['flight'].count().head(10)


,source_city,destination_city,flight
0,Bangalore,Chennai,106
1,Bangalore,Delhi,227
2,Bangalore,Hyderabad,132
3,Bangalore,Kolkata,171
4,Bangalore,Mumbai,175
5,Chennai,Bangalore,69
6,Chennai,Delhi,105
7,Chennai,Hyderabad,82
8,Chennai,Kolkata,110
9,Chennai,Mumbai,94


In [10]:
df.groupby(['airline','source_city','destination_city'],as_index=False)['price'].mean().head(10)


,airline,source_city,destination_city,price
0,AirAsia,Bangalore,Chennai,2073.043478
1,AirAsia,Bangalore,Delhi,4807.092426
2,AirAsia,Bangalore,Hyderabad,2931.494792
3,AirAsia,Bangalore,Kolkata,4443.468160
4,AirAsia,Bangalore,Mumbai,3342.385350
5,AirAsia,Chennai,Bangalore,1914.760870
6,AirAsia,Chennai,Delhi,3697.314003
7,AirAsia,Chennai,Hyderabad,2053.182540
8,AirAsia,Chennai,Kolkata,3682.338762
9,AirAsia,Chennai,Mumbai,2691.100000


In [11]:
df_bk=df.copy()
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in df.columns:
    if df[col].dtype=='object':
        df[col]=le.fit_transform(df[col])

In [12]:
x=df.drop(['price'],axis=1)
y=df['price']

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((210107, 10), (90046, 10), (210107,), (90046,))

In [14]:
from sklearn.preprocessing import MinMaxScaler
mmscaler=MinMaxScaler(feature_range=(0,1))
x_train=mmscaler.fit_transform(x_train)
x_test=mmscaler.fit_transform(x_test)
x_train=pd.DataFrame(x_train)
x_test=pd.DataFrame(x_test)  

In [15]:
a={'Model Name':[], 'Mean_Absolute_Error_MAE':[] ,'Adj_R_Square':[] ,'Root_Mean_Squared_Error_RMSE':[] ,'Mean_Absolute_Percentage_Error_MAPE':[] ,'Mean_Squared_Error_MSE':[] ,'Root_Mean_Squared_Log_Error_RMSLE':[] ,'R2_score':[]}
Results=pd.DataFrame(a)
Results.head()

,Model Name,Mean_Absolute_Error_MAE,Adj_R_Square,Root_Mean_Squared_Error_RMSE,Mean_Absolute_Percentage_Error_MAPE,Mean_Squared_Error_MSE,Root_Mean_Squared_Log_Error_RMSLE,R2_score


In [17]:
from sklearn.linear_model import LinearRegression, Ridge
import numpy as np
from sklearn import metrics

model_linear = LinearRegression()
model_ridge = Ridge()

results = []

for model in [model_linear, model_ridge]:
    model.fit(x_train, y_train)
    
    y_pred = model.predict(x_test)
    
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = metrics.r2_score(y_test, y_pred)
    rmsle = np.log(np.sqrt(mse))
    
    def MAPE(y_test, y_pred):
        y_test, y_pred = np.array(y_test), np.array(y_pred)
        return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mape = MAPE(y_test, y_pred)
    
    r_squared = r2
    adjusted_r_squared = 1 - (1 - r_squared) * (len(y_test) - 1) / (len(y_test) - x_test.shape[1] - 1)
    
    print('Model Name:', type(model).__name__)
    print('Mean Absolute Error (MAE):', round(mae, 3))
    print('Mean Squared Error (MSE):', round(mse, 3))
    print('Root Mean Squared Error (RMSE):', round(rmse, 3))
    print('R2_score:', round(r2, 6))
    print('Root Mean Squared Log Error (RMSLE):', round(rmsle, 3))
    print('Mean Absolute Percentage Error (MAPE):', round(mape, 2), '%')
    print('Adjusted R Squared:', round(adjusted_r_squared, 6))
    print('------------------------------------------------------------------------------------------------------------')

    results.append({
        'Model Name': type(model).__name__,
        'Mean Absolute Error (MAE)': mae,
        'Mean Squared Error (MSE)': mse,
        'Root Mean Squared Error (RMSE)': rmse,
        'R2 Score': r2,
        'Root Mean Squared Log Error (RMSLE)': rmsle,
        'Mean Absolute Percentage Error (MAPE)': mape,
        'Adjusted R Squared': adjusted_r_squared
    })


Model Name: LinearRegression
Mean Absolute Error (MAE): 4637.917
Mean Squared Error (MSE): 49259002.429
Root Mean Squared Error (RMSE): 7018.476
R2_score: 0.904761
Root Mean Squared Log Error (RMSLE): 8.856
Mean Absolute Percentage Error (MAPE): 43.58 %
Adjusted R Squared: 0.904751
------------------------------------------------------------------------------------------------------------
Model Name: Ridge
Mean Absolute Error (MAE): 4637.939
Mean Squared Error (MSE): 49259016.4
Root Mean Squared Error (RMSE): 7018.477
R2_score: 0.904761
Root Mean Squared Log Error (RMSLE): 8.856
Mean Absolute Percentage Error (MAPE): 43.58 %
Adjusted R Squared: 0.904751
------------------------------------------------------------------------------------------------------------


In [20]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn import metrics

# Create objects of Linear Regression, Ridge Regression, and Random Forest Regressor models
model_linear = LinearRegression()
model_ridge = Ridge()
model_rfr = RandomForestRegressor()

# Evaluation matrix for the algorithms
results = []

for model in [model_linear, model_ridge, model_rfr]:
    # Fit the model with train data
    model.fit(x_train, y_train)
    
    # Predict the model with test data
    y_pred = model.predict(x_test)
    
    # Evaluation metrics for Regression analysis
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = metrics.r2_score(y_test, y_pred)
    rmsle = np.log(np.sqrt(mse))
    
    # Define the function to calculate the MAPE - Mean Absolute Percentage Error
    def MAPE(y_test, y_pred):
        y_test, y_pred = np.array(y_test), np.array(y_pred)
        return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mape = MAPE(y_test, y_pred)
    
    # Calculate Adjusted R squared values 
    r_squared = r2
    adjusted_r_squared = 1 - (1 - r_squared) * (len(y_test) - 1) / (len(y_test) - x_test.shape[1] - 1)
    
    # Print the model name and evaluation metrics
    print('Model Name:', type(model).__name__)
    print('Mean Absolute Error (MAE):', round(mae, 3))
    print('Mean Squared Error (MSE):', round(mse, 3))
    print('Root Mean Squared Error (RMSE):', round(rmse, 3))
    print('R2_score:', round(r2, 6))
    print('Root Mean Squared Log Error (RMSLE):', round(rmsle, 3))
    print('Mean Absolute Percentage Error (MAPE):', round(mape, 2), '%')
    print('Adjusted R Squared:', round(adjusted_r_squared, 6))
    print('------------------------------------------------------------------------------------------------------------')

    # Store the results
    results.append({
        'Model Name': type(model).__name__,
        'Mean Absolute Error (MAE)': mae,
        'Mean Squared Error (MSE)': mse,
        'Root Mean Squared Error (RMSE)': rmse,
        'R2 Score': r2,
        'Root Mean Squared Log Error (RMSLE)': rmsle,
        'Mean Absolute Percentage Error (MAPE)': mape,
        'Adjusted R Squared': adjusted_r_squared
    })


Model Name: LinearRegression
Mean Absolute Error (MAE): 4637.917
Mean Squared Error (MSE): 49259002.429
Root Mean Squared Error (RMSE): 7018.476
R2_score: 0.904761
Root Mean Squared Log Error (RMSLE): 8.856
Mean Absolute Percentage Error (MAPE): 43.58 %
Adjusted R Squared: 0.904751
------------------------------------------------------------------------------------------------------------
Model Name: Ridge
Mean Absolute Error (MAE): 4637.939
Mean Squared Error (MSE): 49259016.4
Root Mean Squared Error (RMSE): 7018.477
R2_score: 0.904761
Root Mean Squared Log Error (RMSLE): 8.856
Mean Absolute Percentage Error (MAPE): 43.58 %
Adjusted R Squared: 0.904751
------------------------------------------------------------------------------------------------------------
Model Name: RandomForestRegressor
Mean Absolute Error (MAE): 1188.502
Mean Squared Error (MSE): 8441595.038
Root Mean Squared Error (RMSE): 2905.442
R2_score: 0.983679
Root Mean Squared Log Error (RMSLE): 7.974
Mean Absolute Perc